In [ ]:
import os
import io
import json
import time
import math
import string 
import pickle
import datetime
import itertools
import numpy as np
import pandas as pd
from pprint import pprint 
from tqdm.notebook import tqdm
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
from keybert import KeyBERT

In [ ]:
df_talks = pd.read_csv('../Data/talks_data.csv')

df_talks.head()

In [ ]:
kw_model = KeyBERT(model='paraphrase-MiniLM-L6-v2')

In [ ]:
df_talks['text'] =  df_talks['transcript'] + ' ' + df_talks['description'] + ' ' + df_talks['title']

In [ ]:
keywords = {}
for i, e in tqdm(df_talks.iterrows()):
    doc = e['text']
    talk = e['id']
    kws = kw_model.extract_keywords(doc)
    keywords[talk] = kws

In [ ]:
pickle.dump(keywords, open('keywords.pickle', 'wb'))

In [ ]:
len(keywords)

In [ ]:
keywords = pickle.load(open('keywords.pickle', 'rb'))

In [ ]:
all_keywords = [kw[0] for kws in keywords.values() for kw in kws]

In [ ]:
kw_counter = Counter(all_keywords)

In [ ]:
kw_counter.most_common(10)

In [ ]:
with open('../kg_embeddings/metadata-interactions-kw/ted-kw.txt', 'w') as f:
    for talk in tqdm(df_talks['id'].values):
        talk_keywords = [kw[0] for kw in keywords[talk] if kw_counter[kw[0]] > 1 and kw[1] > 0.3]
        for kw in talk_keywords:
            f.write(f'{talk}\thasKeyword\t{kw}\n')